In [11]:
from torch import optim
from pytorch_lightning import Trainer
import torch
import argparse
import yaml
import torch.nn as nn
from dataloader import create_dataloader
from tools_for_model import near_avg_index, max_index, min_index, Bar, cal_pesq, cal_stoi
from pprint import pprint
from framework import ABF, MultiResolutionSTFTLoss, SPKDLoss, build_review_kd, ReviewKD
from pprint import pprint
from asteroid.utils import prepare_parser_from_dict, parse_args_as_dict
# import inspect module
import inspect
import feature_extraction
import DCCRN
import torch
from asteroid.data import DNSDataset
import os
import torch.nn.functional as F
import time
import torch.nn as nn
import shutil
import numpy as np
from framework import ABF, MultiResolutionSTFTLoss, SPKDLoss, build_review_kd, ReviewKD
from asteroid.models import DCCRNet
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
import config as cfg
from tools_for_model import near_avg_index, max_index, min_index, Bar, cal_pesq, cal_stoi
from dataloader import create_dataloader
from asteroid.metrics import get_metrics
import inspect
import os
import feature_extraction
from asteroid.models import DCCRNet_mini
from asteroid.metrics import get_metrics
from asteroid.data.librimix_dataset import LibriMix
from asteroid.losses import PITLossWrapper, pairwise_neg_sisdr
from asteroid import DCCRNet
from asteroid.models import save_publishable
from asteroid.utils import tensors_to_device
from asteroid.dsp.normalization import normalize_estimates
from asteroid.metrics import WERTracker, MockWERTracker

#os.environ["CUDA_VISIBLE_DEVICES"]="2"



In [ ]:
from asteroid.models import DCCRNet, DCCRNet_mini

In [ ]:
from asteroid.data import DNSDataset
from asteroid.losses import PITLossWrapper, pairwise_neg_sisdr

In [2]:
data_set =  DNSDataset("/root/NTH_student/test_loader")

In [3]:
train_loader = create_dataloader(mode='valid',dataset=data_set)
sisdr = PITLossWrapper(pairwise_neg_sisdr, pit_from="pw_mtx")

In [4]:
X,Y = 0,0
i = 0
for x,y,_ in Bar(train_loader):
    if i == 1: break
    X=x
    Y=y
    i += 1


 64/600: [==>...............................................] - ETA 1.0s

In [5]:
#model_1 = DCCRN.DCCRN(rnn_units=cfg.rnn_units_student, masking_mode=cfg.masking_mode, use_clstm=cfg.use_clstm,kernel_num=cfg.kernel_num_student)
model_2 = DCCRNet.from_pretrained('JorisCos/DCCRNet_Libri1Mix_enhsingle_16k')
#model = DCCRNet_mini(architecture='DCCRN-CL-mini')\
#model_2 = DCCRNet.load_from_checkpoint()


In [25]:
device = torch.device('cpu')

In [26]:
state_dict_1 = torch.load('/root/NTH_student/Speech_Enhancement_new/knowledge_distillation_CLSKD/checkpoint/epoch=72-step=231775.ckpt',
                        map_location=device)
state_dict_2 = torch.load('/root/NTH_student/Speech_Enhancement_new/knowledge_distillation_CLSKD/checkpoint/best_model.pth',
                        map_location=device)

In [28]:
len(state_dict_1['state_dict'])

182

In [166]:
count = 0
for i,j in zip(state_dict_1['state_dict'].items(),state_dict_2['state_dict'].items()):
    if torch.all(i[1]==j[1]) and i[0]==j[0]:
        count += 1
if count == len(state_dict_1['state_dict']):print('true')

In [167]:
modified_state_dict = {key.replace('model.', ''): value for key, value in state_dict_1['state_dict'].items()}
state_dict_1['state_dict'] = modified_state_dict

In [168]:
state_dict_1['state_dict']

{'encoder.filterbank._filters': tensor([[[ 0.0000e+00,  2.4543e-04,  4.9085e-04,  ...,  7.3624e-04,
            4.9085e-04,  2.4543e-04]],
 
         [[ 0.0000e+00,  2.6559e-04,  5.2563e-04,  ...,  7.8000e-04,
            5.2563e-04,  2.6559e-04]],
 
         [[ 0.0000e+00,  5.9340e-05,  1.0186e-04,  ...,  1.2745e-04,
            1.0186e-04,  5.9340e-05]],
 
         ...,
 
         [[ 0.0000e+00, -3.4199e-04,  6.8666e-04,  ...,  1.0334e-03,
           -6.8666e-04,  3.4199e-04]],
 
         [[ 0.0000e+00, -2.2347e-04,  4.5341e-04,  ...,  6.8971e-04,
           -4.5341e-04,  2.2347e-04]],
 
         [[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
            0.0000e+00,  0.0000e+00]]]),
 'encoder.filterbank.torch_window': tensor([0.0000, 0.0079, 0.0157, 0.0236, 0.0314, 0.0393, 0.0471, 0.0550, 0.0628,
         0.0706, 0.0785, 0.0863, 0.0941, 0.1019, 0.1097, 0.1175, 0.1253, 0.1331,
         0.1409, 0.1487, 0.1564, 0.1642, 0.1719, 0.1797, 0.1874, 0.1951, 0.2028,
         0.2105

In [170]:
count = 0
for i,j in zip(state_dict_1['state_dict'].items(),state_dict_2['state_dict'].items()):
    if i[1].shape==j[1].shape and i[0]==j[0]:
        count += 1
        print('TRUE')
    else: print('FALSE')

if count == len(state_dict_1['state_dict']):
    print('true')
else: print('False')

TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
true


In [82]:
model_1 = DCCRNet(architecture='DCCRN-CL-mini')
model_1.load_state_dict(state_dict=state_dict_1["state_dict"])


<All keys matched successfully>

In [83]:
model_2 = DCCRNet(architecture='DCCRN-CL-mini')
model_2.load_state_dict(state_dict=state_dict_2["state_dict"])

<All keys matched successfully>

In [112]:
dict_1 = model_1.get_state_dict()

In [110]:
dict_2 = model_2.get_state_dict()

In [104]:
h = model_1(X[0])

In [105]:
k = model_2(X[0])

In [118]:
for i,j in zip(dict_1.items(),dict_2.items()):
    if torch.all(i[1]==j[1]) and i[0]==j[0]:
        print("TRUE")

TRUE
TRUE
TRUE
TRUE


In [78]:
model = DCCRNet_mini(architecture='DCCRN-CL-test')
state_dict = torch.load('/root/NTH_student/Speech_Enhancement_new/knowledge_distillation_CLSKD/checkpoint/model-epoch=00-si_sdr=-43.81-stoi=0.17.ckpt',
                        map_location=device)

In [79]:
model.cpu()

DCCRNet_mini(
  (encoder): Encoder(
    (filterbank): STFTFB()
  )
  (masker): DCCRMaskNet_mini(
    (encoders): ModuleList(
      (0): DCUNetComplexEncoderBlock(
        (conv): ComplexMultiplicationWrapper(
          (re_module): Conv2d(1, 4, kernel_size=(5, 2), stride=(2, 1), padding=(2, 0), bias=False)
          (im_module): Conv2d(1, 4, kernel_size=(5, 2), stride=(2, 1), padding=(2, 0), bias=False)
        )
        (norm): OnReIm(
          (re_module): BatchNorm(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (im_module): BatchNorm(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (activation): OnReIm(
          (re_module): PReLU(num_parameters=1)
          (im_module): PReLU(num_parameters=1)
        )
      )
      (1): DCUNetComplexEncoderBlock(
        (conv): ComplexMultiplicationWrapper(
          (re_module): Conv2d(4, 8, kernel_size=(5, 2), stride=(2, 1), padding=(2, 0), bias=False)
          (im_module): C

In [80]:
modified = {}
for key,value in state_dict['state_dict'].items():
    if key.startswith('student.'):
        modified[key.replace('student.','')] = value
    else:
        continue
state_dict['state_dict'] = modified

In [81]:
len(state_dict['state_dict'])

182

In [82]:
model.load_state_dict(state_dict=state_dict["state_dict"])


<All keys matched successfully>

In [84]:
to_save = model.serialize()
#torch.save(to_save,os.path.join('/root/NTH_student/Speech_Enhancement_new/knowledge_distillation_CLSKD/checkpoint', "tgt_model.pth"))

In [77]:
mod = DCCRNet_mini.from_pretrained('/root/NTH_student/Speech_Enhancement_new/knowledge_distillation_CLSKD/checkpoint/tgt_model.pth')
mod.state_dict

<bound method Module.state_dict of DCCRNet_mini(
  (encoder): Encoder(
    (filterbank): STFTFB()
  )
  (masker): DCCRMaskNet_mini(
    (encoders): ModuleList(
      (0): DCUNetComplexEncoderBlock(
        (conv): ComplexMultiplicationWrapper(
          (re_module): Conv2d(1, 4, kernel_size=(5, 2), stride=(2, 1), padding=(2, 0), bias=False)
          (im_module): Conv2d(1, 4, kernel_size=(5, 2), stride=(2, 1), padding=(2, 0), bias=False)
        )
        (norm): OnReIm(
          (re_module): BatchNorm(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (im_module): BatchNorm(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (activation): OnReIm(
          (re_module): PReLU(num_parameters=1)
          (im_module): PReLU(num_parameters=1)
        )
      )
      (1): DCUNetComplexEncoderBlock(
        (conv): ComplexMultiplicationWrapper(
          (re_module): Conv2d(4, 8, kernel_size=(5, 2), stride=(2, 1), padding=(2, 0), b

In [6]:
X.shape

torch.Size([32, 48000])

In [7]:
pred = model_2(X)

In [9]:
pred.squeeze(1).shape

torch.Size([32, 48000])

In [29]:
# _,_,_,_,student_preds = model_1(X)

# mix = X.cpu().data.numpy()
# clean= y.cpu().data.numpy()
# estimate = student_preds.cpu().squeeze(1).data.numpy()
# metric_dict = get_metrics(mix=mix,clean=clean,estimate=estimate)
# metric_dict

In [ ]:
device = torch.device('cpu')

In [31]:
pred = model_2(X[0])

# mix = X[0].cpu().data.numpy()
# clean= y[0].cpu().data.numpy()
# estimate = pred.squeeze(1).cpu().data.numpy()
# metric_dict = get_metrics(mix=mix,clean=clean,estimate=estimate)
# metric_dict

mix, sources, ids = X[0]
mix, sources = tensors_to_device([mix, sources], device=device)
est_sources = model(mix.unsqueeze(0))
loss, reordered_sources = loss_func(est_sources, sources[None], return_est=True)
mix_np = mix.cpu().data.numpy()
sources_np = sources.cpu().data.numpy()
est_sources_np = reordered_sources.squeeze(0).cpu().data.numpy()
# For each utterance, we get a dictionary with the mixture path,
# the input and output metrics
utt_metrics = get_metrics(
    mix_np,
    sources_np,
    est_sources_np,
    sample_rate=conf["sample_rate"],
    metrics_list=COMPUTE_METRICS,
    )

{'input_si_sdr': -39.185794830322266,
 'input_sdr': -19.40019439764415,
 'input_sir': inf,
 'input_sar': -19.40019439764415,
 'input_stoi': 0.2039877502314637,
 'input_pesq': 1.1211283206939697,
 'si_sdr': -39.599822998046875,
 'sdr': -20.8135003071654,
 'sir': inf,
 'sar': -20.8135003071654,
 'stoi': 0.06268259324754781,
 'pesq': 1.0211503505706787}

In [36]:
loss_func = PITLossWrapper(pairwise_neg_sisdr, pit_from="pw_mtx")

In [42]:
mix = X[0]
sources = y[0]
#mix, sources = tensors_to_device([mix, sources], device=torch.device('cpu'))
est_sources = model_2(mix.unsqueeze(0))
loss, reordered_sources = loss_func(est_sources, sources[None], return_est=True)
mix_np = mix.cpu().data.numpy()
sources_np = sources.cpu().data.numpy()
est_sources_np = reordered_sources.squeeze(0).cpu().data.numpy()
# For each utterance, we get a dictionary with the mixture path,
# the input and output metrics
utt_metrics = get_metrics(
mix_np,
sources_np,
est_sources_np,
sample_rate=16000)

AssertionError: Expected source axis along dim 1, found 48000

In [25]:
X[0].unsqueeze(0).shape



torch.Size([1, 48000])

In [13]:
pred = model_2(X[0])

In [14]:
pred.shape

torch.Size([1, 48000])

In [ ]:
from torchinfo import summary
summary(model,(8,48000),device="cpu")

Layer (type:depth-idx)                                            Output Shape              Param #
DCCRNet                                                           [8, 1, 48000]             --
├─Encoder: 1-1                                                    [8, 514, 477]             --
├─DCCRMaskNet: 1-2                                                [8, 1, 256, 477]          --
│    └─ModuleList: 2-1                                            --                        --
│    │    └─DCUNetComplexEncoderBlock: 3-1                        [8, 16, 128, 476]         386
│    │    └─DCUNetComplexEncoderBlock: 3-2                        [8, 32, 64, 475]          10,370
│    │    └─DCUNetComplexEncoderBlock: 3-3                        [8, 64, 32, 474]          41,218
│    │    └─DCUNetComplexEncoderBlock: 3-4                        [8, 128, 16, 473]         164,354
│    │    └─DCUNetComplexEncoderBlock: 3-5                        [8, 128, 8, 472]          328,194
│    │    └─DCUNetComplexE

In [9]:
model.masker

DCCRMaskNet_mini(
  (encoders): ModuleList(
    (0): DCUNetComplexEncoderBlock(
      (conv): ComplexMultiplicationWrapper(
        (re_module): Conv2d(1, 8, kernel_size=(5, 2), stride=(2, 1), padding=(2, 0), bias=False)
        (im_module): Conv2d(1, 8, kernel_size=(5, 2), stride=(2, 1), padding=(2, 0), bias=False)
      )
      (norm): OnReIm(
        (re_module): BatchNorm(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (im_module): BatchNorm(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (activation): OnReIm(
        (re_module): PReLU(num_parameters=1)
        (im_module): PReLU(num_parameters=1)
      )
    )
    (1): DCUNetComplexEncoderBlock(
      (conv): ComplexMultiplicationWrapper(
        (re_module): Conv2d(8, 16, kernel_size=(5, 2), stride=(2, 1), padding=(2, 0), bias=False)
        (im_module): Conv2d(8, 16, kernel_size=(5, 2), stride=(2, 1), padding=(2, 0), bias=False)
      )
      (norm): OnReIm(
        (r

In [10]:
pprint(inspect.getmembers(model))

[('T_destination', ~T_destination),
 ('_BaseModel__sample_rate', 16000.0),
 ('__annotations__',
  {'__call__': typing.Callable[..., typing.Any],
   '_is_full_backward_hook': typing.Optional[bool],
   '_version': <class 'int'>,
   'dump_patches': <class 'bool'>,
   'forward': typing.Callable[..., typing.Any],
   'training': <class 'bool'>}),
 ('__call__',
  <bound method Module._call_impl of DCCRNet_mini(
  (encoder): Encoder(
    (filterbank): STFTFB()
  )
  (masker): DCCRMaskNet_mini(
    (encoders): ModuleList(
      (0): DCUNetComplexEncoderBlock(
        (conv): ComplexMultiplicationWrapper(
          (re_module): Conv2d(1, 8, kernel_size=(5, 2), stride=(2, 1), padding=(2, 0), bias=False)
          (im_module): Conv2d(1, 8, kernel_size=(5, 2), stride=(2, 1), padding=(2, 0), bias=False)
        )
        (norm): OnReIm(
          (re_module): BatchNorm(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (im_module): BatchNorm(8, eps=1e-05, momentum=0.1, affi

In [11]:
outputs_enc = []
outputs_dec= []
output_lstm = []

def hook_enc(module, input, output):
    outputs_enc.append(output.detach().cpu()) 
def hook_dec(module, input, output):
    outputs_dec.append(output.detach().cpu())
def hook_lstm(module, input, output):
    output_lstm.append(output.detach())


In [12]:
#handler_enc = [model.masker.encoders[i].register_forward_hook(hook_enc) for i in range(len(model.masker.encoders)-1)]
#handler_dec = [model.masker.decoders[i].register_forward_hook(hook_dec) for i in range(len(model.masker.decoders))]

In [13]:
handler_enc = model.masker.encoders[0].norm.register_forward_hook(hook_enc)
handler_lstm = model.masker.encoders[-1].rnn.rnns[1].register_forward_hook(hook_lstm)

In [14]:
model.masker.encoders[0].norm

OnReIm(
  (re_module): BatchNorm(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (im_module): BatchNorm(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [15]:
model.masker.encoders[-1]

DCCRMaskNetRNN_mini(
  (rnn): ComplexSingleRNN(
    (rnns): ModuleList(
      (0): ComplexMultiplicationWrapper(
        (re_module): SingleRNN(
          (rnn): LSTM(256, 32, batch_first=True)
        )
        (im_module): SingleRNN(
          (rnn): LSTM(256, 32, batch_first=True)
        )
      )
      (1): ComplexMultiplicationWrapper(
        (re_module): SingleRNN(
          (rnn): LSTM(32, 32, batch_first=True)
        )
        (im_module): SingleRNN(
          (rnn): LSTM(32, 32, batch_first=True)
        )
      )
    )
  )
  (linear): ComplexMultiplicationWrapper(
    (re_module): Linear(in_features=32, out_features=256, bias=True)
    (im_module): Linear(in_features=32, out_features=256, bias=True)
  )
)

In [22]:
# model(X)

tensor([[[ 0.0000e+00, -2.8173e-05,  1.2323e-04,  ..., -6.9996e-05,
           3.5402e-05,  1.1099e-04]],

        [[ 0.0000e+00,  6.2609e-05,  1.5794e-04,  ...,  6.4208e-05,
           4.6206e-05,  2.1527e-05]],

        [[ 0.0000e+00,  1.2102e-04,  2.3302e-04,  ...,  8.0085e-05,
          -1.0738e-04, -1.0527e-04]],

        ...,

        [[ 0.0000e+00, -7.4628e-05, -1.5103e-04,  ...,  3.8449e-04,
           2.7326e-04,  1.4773e-04]],

        [[ 0.0000e+00,  4.5428e-07,  7.7428e-07,  ...,  1.0771e-04,
           7.7169e-05,  4.8407e-05]],

        [[ 0.0000e+00,  1.2885e-06,  7.5364e-06,  ...,  4.0597e-05,
           1.8846e-06, -7.5834e-06]]], grad_fn=<ConstantPadNdBackward0>)

In [17]:
outputs_enc[0].type()

'torch.ComplexFloatTensor'

In [18]:
for i in range(len(outputs_enc)):
    print(outputs_enc[i].shape)
print('\n')
for i in range(len(output_lstm)):
    print(output_lstm[i].shape)
print('\n')
for i in range(len(outputs_dec)):
    print(outputs_dec[i].shape)

torch.Size([32, 8, 128, 476])


torch.Size([32, 471, 32])




# encoder

In [19]:
for i in range(len(outputs_enc)):
    print(outputs_enc[i].shape)

torch.Size([32, 8, 128, 476])


In [20]:
in_channels = [8, 16, 32, 64, 64, 64]
out_channels = [32, 64, 128, 256, 256, 256]

# shapes = [1,4,8,16,32]
# out_shapes = [1,4,8,16,32]
shapes = [4,8,16,32,64,128]
out_shapes = [4,8,16,32,64,128]
abfs = nn.ModuleList()
mid_channel = min(512, in_channels[-1])
for idx, in_channel in enumerate(in_channels):
    abfs.append(ABF(in_channel, mid_channel, out_channels[idx], idx < len(in_channels)-1))
abfs = abfs[::-1]

In [21]:
x = outputs_enc
results = []
out_features, res_features = abfs[0](x[0].cuda(), out_shape=out_shapes[0],feature_type="decoder")
results.append(out_features)
for feature, abf, shape, out_shape in zip(x[1:], abfs[1:], shapes[1:], out_shapes[1:]):
        out_features, res_features = abf(feature.cuda(), res_features, shape, out_shape,feature_type="decoder")
        print("1")
        results.append(out_features)

RuntimeError: Input type (CUDAComplexFloatType) and weight type (torch.cuda.FloatTensor) should be the same

# decoder

In [ ]:
for i in range(len(outputs_dec)):
    print(outputs_dec[i].shape)

In [ ]:
handler_enc.remove()
handler_dec.remove()